In [3]:
# 필요한 라이브러리 설치
!pip install transformers

In [4]:
# transformers 라이브러리에서 필요한 모듈 임포트
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# 모델과 토크나이저 로드
model_name = "pongjin/roberta_with_kornli"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/985 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/415 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [5]:
# 파이프라인 생성
nli_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [6]:
# 파이프라인 사용 예시
text = "여기가 정말 좋아요!"
result = nli_pipeline(text)
print(result)

[{'label': 'entailment', 'score': 0.9272531270980835}]


In [7]:
class ArgumentHandler():
    """
    Base interface for handling arguments for each :class:`~transformers.pipelines.Pipeline`.
    """
    def __call__(self, *args, **kwargs):
        raise NotImplementedError()


class CustomZeroShotClassificationArgumentHandler(ArgumentHandler):
    """
    Handles arguments for zero-shot for text classification by turning each possible label into an NLI
    premise/hypothesis pair.
    """

    def _parse_labels(self, labels):
        if isinstance(labels, str):
            labels = [label.strip() for label in labels.split(",")]
        return labels

    def __call__(self, sequences, labels, hypothesis_template):
        if len(labels) == 0 or len(sequences) == 0:
            raise ValueError("You must include at least one label and at least one sequence.")
        if hypothesis_template.format(labels[0]) == hypothesis_template:
            raise ValueError(
                (
                    'The provided hypothesis_template "{}" was not able to be formatted with the target labels. '
                    "Make sure the passed template includes formatting syntax such as {{}} where the label should go."
                ).format(hypothesis_template)
            )

        if isinstance(sequences, str):
            sequences = [sequences]
        labels = self._parse_labels(labels)

        sequence_pairs = []
        for label in labels:
            # 수정부: 두 문장을 페어로 입력했을 때, `token_type_ids`가 자동으로 붙는 문제를 방지하기 위해 미리 두 문장을 `sep_token` 기준으로 이어주도록 함
            sequence_pairs.append(f"{sequences} {tokenizer.sep_token} {hypothesis_template.format(label)}")

        return sequence_pairs, sequences


In [8]:
classifier = pipeline(
    "zero-shot-classification",
    args_parser=CustomZeroShotClassificationArgumentHandler(),
    model="pongjin/roberta_with_kornli"
)


In [9]:
sequence = "배당락 D-1 코스피, 2330선 상승세...외인·기관 사자"
candidate_labels =["외환",'환율', "경제", "금융", "부동산","주식"]

classifier(
    sequence,
    candidate_labels,
    hypothesis_template='이는 {}에 관한 것이다.',
)

{'sequence': '배당락 D-1 코스피, 2330선 상승세...외인·기관 사자',
 'labels': ['주식', '금융', '경제', '외환', '환율', '부동산'],
 'scores': [0.5052875876426697,
  0.17972509562969208,
  0.1385297030210495,
  0.09460819512605667,
  0.04294908419251442,
  0.03890027105808258]}

In [10]:
sequence = "바질파스타 쉬림프알리오올리오"
candidate_labels = [
    "족발/보쌈",'돈까스', "회", '백반', "죽", "국수",
    "버거", "분식", "국밥", "일식", "고기/구이","피자",
    '치킨', "찜/탕/찌개", "양식", "중식", "아시안"
    ]

text = classifier(
    sequence,
    candidate_labels,
    hypothesis_template='이는 {}에 관한 것이다.',
)

In [11]:
classifier

In [12]:
def getLabel(dish_name) :
    sequence = dish_name
    candidate_labels = [
        "족발/보쌈",'돈까스', "회", '백반', "죽", "국수",
        "버거", "분식", "국밥", "일식", "고기/구이","피자",
        '치킨', "찜/탕/찌개", "양식", "중식", "아시안"
        ]

    text = classifier(
        sequence,
        candidate_labels,
        hypothesis_template='이는 {}에 관한 것이다.',
    )

    return text['labels'][0]

In [13]:
getLabel("피시프레시")

'피자'

In [14]:
# 테스트 끝 본 코드 시작

In [15]:
import pandas as pd
from tqdm import tqdm

In [20]:
# CSV 파일 읽기
df = pd.read_csv('/content/c4_general_cafeteria_chuncheon_opened_SS-HJ_addLatiLongti_menu-price_addedLabel_v2.csv', encoding='utf-8')

labels = [] # 새로운 열을 저장할 리스트

In [17]:
df

,index_cafeteria,인허가일자,영업상태구분코드,소재지전체주소,Latitude,Longitude,cafeteria_name,업태구분명,시설총규모,최종수정시점,시/도,구/군,동/면,상세주소,menu_name,menu_price,label
0,7502,2023-08-16,1,강원특별자치도 춘천시 석사동 654-19,37.859445,127.740166,백돈춘천점,기타,86.13,2023-08-16 14:35:53,강원특별자치도,춘천시,석사동,654-19,[],[],[]
1,7519,2023-08-09,1,강원특별자치도 춘천시 석사동 885-3,37.852339,127.745196,야미마라탕,중국식,96.00,2023-08-11 16:34:22,강원특별자치도,춘천시,석사동,885-3,"['꿔바로우소', '새우볶음밥', '마라탕']",[],"['중식', '찜/탕/찌개', '중식']"
2,7523,2023-08-11,1,강원특별자치도 춘천시 효자동 590-48,37.869167,127.725577,곰소젓갈밥상,한식,87.84,2023-08-11 16:26:26,강원특별자치도,춘천시,효자동,590-48,[],[],[]
3,7524,2023-08-11,1,강원특별자치도 춘천시 효자동 652-3,37.875470,127.735500,호앤호,기타,112.00,2023-08-21 16:31:40,강원특별자치도,춘천시,효자동,652-3,[],[],[]
4,7526,2023-08-04,1,강원특별자치도 춘천시 효자동 305-46,37.864947,127.734955,넘버25호텔남춘천점,기타,53.04,2023-08-07 8:58:17,강원특별자치도,춘천시,효자동,305-46,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,11264,2016-05-03,1,강원특별자치도 춘천시 효자동 618-10,37.874145,127.741954,돌다리야채곱창,기타,95.00,2022-01-20 17:34:08,강원특별자치도,춘천시,효자동,618-10,"['야채곱창', '흑미순대볶음', '흑미순대곱창']","['10,000 ', '12,000...","['분식', '고기/구이', '분식']"
848,11266,2016-08-24,1,강원특별자치도 춘천시 석사동 888-5,37.851051,127.745225,유명한맛돼지,식육(숯불구이),142.55,2022-02-07 17:33:22,강원특별자치도,춘천시,석사동,888-5,"['특목살', '생삼겹살', '가브리살']",[],"['고기/구이', '양식', '회']"
849,11271,2019-05-22,1,강원특별자치도 춘천시 효자동 547,37.871544,127.726458,남부옛날통닭,호프/통닭,21.46,2019-06-04 14:39:14,강원특별자치도,춘천시,효자동,547,[],[],[]
850,11278,2021-08-26,1,강원특별자치도 춘천시 효자동 693,37.868983,127.729208,순살만공격춘천점,호프/통닭,28.50,2022-01-20 11:43:44,강원특별자치도,춘천시,효자동,693,"['닭다리 순살만 4가지맛 세트', '페퍼로니 피자&치킨 세트', '치파포 V5']",[],"['고기/구이', '피자', '치킨']"


In [18]:
# 테스트
i = 1
text = df['menu_name'][i]+" "+df['업태구분명'][i]+" "+df['cafeteria_name'][i]
print("input_text : ", text)
label = getLabel(text)
print("output_result : ", label)

input_text :  ['꿔바로우소', '새우볶음밥', '마라탕'] 중국식 야미마라탕
output_result :  찜/탕/찌개


In [21]:
# tqdm을 사용하여 진행률 표시
for i in tqdm(range(len(df))):
  try:
      text = df['menu_name'][i]+" "+df['업태구분명'][i]+" "+df['cafeteria_name'][i]
      label = getLabel(text)
      print("\ninput_text : ", text)
      print("output_result : ", label)
  except Exception as e:
      print("Error:", e)
      label = -1  # 또는 적절한 기본값
  labels.append(label)

  0%|          | 1/852 [00:02<35:35,  2.51s/it]


input_text :  [] 기타 백돈춘천점
output_result :  백반


  0%|          | 2/852 [00:05<43:16,  3.05s/it]


input_text :  ['꿔바로우소', '새우볶음밥', '마라탕'] 중국식 야미마라탕
output_result :  찜/탕/찌개


  0%|          | 3/852 [00:09<44:01,  3.11s/it]

...


input_text :  [] 호프/통닭 남부옛날통닭
output_result :  치킨


100%|█████████▉| 851/852 [49:52<00:03,  3.67s/it]


input_text :  ['닭다리 순살만 4가지맛 세트', '페퍼로니 피자&치킨 세트', '치파포 V5'] 호프/통닭 순살만공격춘천점
output_result :  고기/구이


100%|██████████| 852/852 [49:57<00:00,  3.52s/it]


input_text :  ['콩나물국밥', '황태콩나물국밥', '콩나물비빔밥'] 한식 한방전주콩나물국밥효자점
output_result :  국밥


In [22]:
# 새로운 열 추가
df['label'] = labels

# 결과를 새로운 CSV 파일로 저장
df.to_csv('/content/c4_general_cafeteria_chuncheon_opened_SS-HJ_addLatiLongti_menu-price_addedLabel_v3.csv', index=False, encoding='utf-8')

In [ ]:
 결과를 새로운 CSV 파일로 저장
df.to_csv('/content/c4_general_cafeteria_chuncheon_opened_SS-HJ_addLatiLongti_menu-price_addedLabel_v3.csv', index=False, encoding='utf-8')